In [1]:
import re
from copy import deepcopy

class OperationNotSupported(Exception):
    # for raising the custom error
    def __init__(self, value):
        super(OperationNotSupported, self).__init__(value)
        
class Relation:
    '''
    Relation in Codd's relational model.
    
    Parameters
    ----------
    schema : [list]
        Schema of the relation.
    instance : [dictionary]
        Instance of the relation.
    '''
    
    def __init__(self, name=None, schema=None, instance=None, dictionary=0):
        self.name = name
        self.schema = schema
        self.instance = instance
        
        # when new Relation object is created
        if dictionary == 0:
            self._datalist = self._create_dict()
            
        # when the return object is empty list and dictionary is 1
        elif dictionary == 1:
            self._datalist = None
            
        # when the return object is not empty list
        else:
            self._datalist = dictionary
            
    # don't want to allow _datalist property to update outside of class
    @property
    def datalist(self):
        return self._datalist
        
    def _create_dict(self):
        
        #this function create list of dictionary records
        #eg. [{'A': 3, 'B': 4, 'C': 5},
        #    {'A': 13, 'B': 2, 'C': 3},
        #    {'A': 5, 'B': 6, 'C': 7}]
        records = []
        
        # check duplicate column name
        if len(self.schema) != len(set(self.schema)):
            raise OperationNotSupported('Duplicate column names')
        
        # check column counts and value counts whether they are matching
        elif max(len(x) for x in self.instance) != len(set(self.schema)):
            raise OperationNotSupported('Column counts and value counts did not match')
                
        else:
            for row in self.instance:
                line = {}
                for col in range(len(self.schema)):
                    line[self.schema[col]] = row[col]
                records.append(line)
            
        return records
    
    def show_result_table(self,dict_set):
        
        #find the max length in both key and value of datalist to display the table as of max column width
        all_rows = []
        for row in dict_set:
            all_rows += list(row.keys())
            all_rows += list(row.values())
            
        lens = [len(str(ele)) for ele in all_rows]
        if lens:
            mm = max(lens)
        else:
            mm = 0
            
        if mm != 0:
            #show results like a table
            result = ''
            rows = ''

            for row in dict_set:
                header = ''
                rows += '\n'
                for i in list(row.keys()):
                    header += '{value:>{length}}|'.format(value=i,length=mm+1)
                    rows += '{value:>{length}}|'.format(value=row[i],length=mm+1)
                header += '\n'
                for i in list(row.keys()):
                    header += '{value:>{length}}|'.format(value='-'*(mm+1),length=mm)

            result += header
            result += rows
            return result
        
    def __repr__(self):
        try:
            if not self._datalist:
                # if datalist is None
                return '0'
            else:
                return self.show_result_table(self._datalist)
        except:
            raise OperationNotSupported('Table cannot be shown')
        
    def select(self, condition):
        '''
        Applies the select operation.
        
        Parameters
        ----------
        condition : str
            Condition for the select operation.
        
        Returns
        -------
        Relation
            The resulting relation.
        
        Raises
        ------
        SyntaxError
            If condition has incorrect syntax.
        '''
        
        result = []
        
        #translate the condition argument into resonable syntax that can be run from python
        
        condition = re.sub(" and ", " ^ ", condition, flags=re.IGNORECASE)
        condition = re.sub(" or ", " | ", condition, flags=re.IGNORECASE)
        condition = re.sub(" not ", " ! ", condition, flags=re.IGNORECASE)
        condition = condition.replace(">=", ">eq")
        condition = condition.replace("<=", "<eq")
        condition = condition.replace("=","==")

        for key in list(self._datalist[0].keys()):
            condition = condition.replace(key,f"row['{key}']")
    
        condition = condition.replace(" ^ ", " and ")
        condition = condition.replace(" | ", " or ")
        condition = condition.replace(" ! ", " not ")
        condition = condition.replace(">eq",">=")
        condition = condition.replace("<eq","<=")

        try:
            for row in self._datalist:
                boo = eval(condition)
                if boo:
                    result.append(row)
            if result:        
                return Relation(dictionary=result)
            else:
                return Relation(dictionary=1)
            
        except: 
            raise OperationNotSupported("Condition is incorrect")
            
    def __getitem__(self, attrs):
        '''
        Applies the project operation.
        
        Parameters
        ----------
        attrs: sequence of str
            Attributes for the project operation.
        
        Returns
        -------
        Relation
            The resulting relation.
        
        Raises
        ------
        KeyError
            If the specified attributes do not exist.
        '''
       
        result = []
        
        for row in self._datalist:
            dd = {}
            for key,value in row.items():
                if key == attrs:
                    dd[key] = value

            result.append(dd)
       
        if result[0]:
            return Relation(dictionary=result)
        else:
            raise OperationNotSupported('No Matching Column Found')
        
        
    def rename(self, mapping):
        '''
        Applies the rename operation.
        
        Parameters
        ----------
        mapping : dict mapping str to str
            A mapping from old names to new names.
        
        Returns
        -------
        Relation
            The resulting relation.
        
        Raises
        ------
        KeyError
            If the specified attributes do not exist.
        '''
        
        # rename({'Oldname':'Newname'})
        
        # check the old column name of argument is in our shema
        for key in mapping.keys():
            if not key in self.schema:
                raise OperationNotSupported('No Matching Existing Columns found')
                
        # check the new column name of argument is equal with the existing column name
        for value in mapping.values():
            if value in self.schema:
                raise OperationNotSupported('Column name already existed')
             
        for key,value in mapping.items():
            result = []
            for row in self._datalist:
                row[value] = row.pop(key)
                result.append(row)
                
            for i in range(len(self.schema)):
                if self.schema[i] == key:
                    self.schema[i] = value

        return Relation(dictionary=result)            
        
    def __or__(self, other):
        '''
        Applies the union operation.
        
        Parameters
        ----------
        other : Relation
            The second operand for the union operation.
        
        Returns
        -------
        Relation
            The resulting relation.
        
        Raises
        ------
        ValueError
            If the schemata do not match.
        '''

        result = []
        
        # create master column name list to access both tables
        keylist = set([key for row in self._datalist for key in row.keys()])
        valuelist = []

        if len(self.schema) == len(other.schema):
        # check the columns of both tables is match or not
            try:
                for row in self._datalist:
                    dd = {}
                    for key in keylist:
                        dd[key] = row[key]
                    result.append(dd)
                    valuelist.append(tuple(dd.values()))

                for row in other._datalist:
                    dd = {}
                    for key in keylist:
                        dd[key] = row[key]
                    if tuple(dd.values()) not in valuelist:
                        result.append(dd)
                        
                return Relation(dictionary=result)
            
            except:
                raise OperationNotSupported('Cannot union different schema')
        
        else:
            raise OperationNotSupported('Column counts not match')
        
    def __sub__(self, other):
        '''
        Applies the difference operation.
        
        Parameters
        ----------
        other : Relation
            The second operand for the difference operation.
        
        Returns
        -------
        Relation
            The resulting relation.
        
        Raises
        ------
        ValueError
            If the schemata do not match.
        '''
        
        result = []
        
        # create master column name list to access both tables
        keylist = set([key for row in self._datalist for key in row.keys()])
        valuelist = []
        
        if len(self.schema) == len(other.schema):
        # check the columns of both tables is match or not
            try:
                for row in other._datalist:
                    dd = {}
                    for key in keylist:
                        dd[key] = row[key]
                    valuelist.append(tuple(dd.values()))

                for row in self._datalist:
                    dd = {}
                    for key in keylist:
                        dd[key] = row[key]
                    if tuple(dd.values()) not in valuelist:
                        result.append(dd)

                if result:        
                    return Relation(dictionary=result)
                else:
                    return Relation(dictionary=1)
                
            except:
                raise OperationNotSupported('Cannot substract different schema')
        else:
            raise OperationNotSupported('Column counts not match')
        
    def __mul__(self, other):
        '''
        Applies the Cartesian product operation.
        
        Parameters
        ----------
        other: Relation
            The second operand for the Cartesian product operation.
        
        Returns
        -------
        Relation
            The resulting relation.
        '''
        
        #clone the original list to prevent changing column name
        slf = deepcopy(self._datalist)
        othr = deepcopy(other._datalist)
        
        #rename the same column names of two tables as TableName.ColumnName
        akeylist = set([key for row in slf for key in row.keys()])
        bkeylist = set([key for row in othr for key in row.keys()])

        samekey = akeylist.intersection(bkeylist)

        for arow in slf:
            for key in arow.keys():
                if key in samekey:
                    arow[self.name+'.'+key] = arow.pop(key)

        for brow in othr:
            for key in brow.keys():
                if key in samekey:
                    brow[other.name+'.'+key] = brow.pop(key)

        result = []

        # generate Cartesian product
        for arow in slf:
            dd = {}
            for akey,avalue in arow.items():
                dd[akey] = avalue
            for brow in othr:
                for bkey,bvalue in brow.items():
                    dd[bkey] = bvalue
                result.append(dict(dd))

        return Relation(dictionary=result)

### Create Relation 'R' and 'S'

Notice : Relation object retrun a string displaying the relation as a table when calling as <b>str</b> or <b>repr</b> object. You can also get the data of relation as list of dictionaries when you call <b>datalist</b> properties.

In [3]:
R = Relation('R',['A','B'],{(1,2),(3,4)})
S = Relation('S',['C','D'],{(5,6),(3,4)})

In [4]:
print(R)
R.datalist

 A| B|
--|--|
 1| 2|
 3| 4|


[{'A': 1, 'B': 2}, {'A': 3, 'B': 4}]

In [5]:
print(S)
S.datalist

 C| D|
--|--|
 5| 6|
 3| 4|


[{'C': 5, 'D': 6}, {'C': 3, 'D': 4}]

### 1. Selection ($\sigma$)

In [6]:
print(R.select('B=2'))
R.select('B=2').datalist

 A| B|
--|--|
 1| 2|


[{'A': 1, 'B': 2}]

In [7]:
print(R.select('not B=2'))
R.select('not B=2').datalist

 A| B|
--|--|
 3| 4|


[{'A': 3, 'B': 4}]

In [8]:
print(R.select('B>3'))
R.select('B>3').datalist

 A| B|
--|--|
 3| 4|


[{'A': 3, 'B': 4}]

In [9]:
print(R.select('B<3'))
R.select('B<3').datalist

 A| B|
--|--|
 1| 2|


[{'A': 1, 'B': 2}]

In [10]:
print(R.select('A>=3'))
R.select('A>=3').datalist

 A| B|
--|--|
 3| 4|


[{'A': 3, 'B': 4}]

In [11]:
print(R.select('A<=3'))
R.select('A<=3').datalist

 A| B|
--|--|
 1| 2|
 3| 4|


[{'A': 1, 'B': 2}, {'A': 3, 'B': 4}]

In [12]:
print(R.select('A=1 and B=2'))
R.select('A=1 and B=2').datalist

 A| B|
--|--|
 1| 2|


[{'A': 1, 'B': 2}]

In [13]:
print(R.select('A=1 or B=4'))
R.select('A=1 or B=4').datalist

 A| B|
--|--|
 1| 2|
 3| 4|


[{'A': 1, 'B': 2}, {'A': 3, 'B': 4}]

Return str object to <b>0</b> and datalist property to <b>None</b> when no results are found.

In [14]:
print(R.select('B=0'))
print(R.select('B=0').datalist)

0
None


Raise <b>Condition is incorrect</b> error when the parsed condition is not correct.

In [15]:
R.select('C=2')
R.select('B==4')

OperationNotSupported: Condition is incorrect

### 2. Projection ($\pi$)

In [16]:
print('R')
print(R)

R
 A| B|
--|--|
 1| 2|
 3| 4|


In [17]:
print(R['A'])
R['A'].datalist

 A|
--|
 1|
 3|


[{'A': 1}, {'A': 3}]

In [18]:
print(R['B'])
R['B'].datalist

 B|
--|
 2|
 4|


[{'B': 2}, {'B': 4}]

Raise <b>No Matching Column Found</b> error when the parsed column name is not in the table.

In [19]:
print(R['C'])
R['C'].datalist

OperationNotSupported: No Matching Column Found

### 3. Rename ($\rho$)

In [20]:
R = Relation('R',['A','B'],{(1,2),(3,4)})
print('R')
print(R)

R
 A| B|
--|--|
 1| 2|
 3| 4|


In [21]:
R.rename({'A':'AA','B':'BB'})

print('R')
print(R)
R.datalist

R
 AA| BB|
---|---|
  1|  2|
  3|  4|


[{'AA': 1, 'BB': 2}, {'AA': 3, 'BB': 4}]

Raise <b>No Matching Existing Columns found</b> error when the parsed old column name is not in the table.

In [22]:
R.rename({'C':'AA','D':'BB'})

OperationNotSupported: No Matching Existing Columns found

Raise <b>Column name already existed</b> error when the parsed new column name is already existed in the table.

In [23]:
R.rename({'BB':'AA'})

OperationNotSupported: Column name already existed

### 4. Union ($\cup$)

In [24]:
R = Relation('R',['A','B'],{(1,2),(3,4)})
S = Relation('S',['C','D'],{(5,6),(3,4)})
print('R')
print(R)
print('\nS')
print(S)

R
 A| B|
--|--|
 1| 2|
 3| 4|

S
 C| D|
--|--|
 5| 6|
 3| 4|


Raise <b>Cannot union different schema</b> error when the two tables have different column names.

In [25]:
(R | S)

OperationNotSupported: Cannot union different schema

Change the column name of relation <b>S</b> into <b>A</b> and <b>B</b> to be matched with relation <b>R</b>.

In [26]:
S.rename({'C':'A','D':'B'})
print('S')
print(S)

S
 A| B|
--|--|
 5| 6|
 3| 4|


In [27]:
print(R | S)
(R | S).datalist

 A| B|
--|--|
 1| 2|
 3| 4|
 5| 6|


[{'A': 1, 'B': 2}, {'A': 3, 'B': 4}, {'A': 5, 'B': 6}]

Add new column to relation <b>R</b>. Now relation <b>R</b> have <b>3 columns</b> and relation <b>S</b> have only <b>2 columns</b>.

In [28]:
R = Relation('R',['A','B','C'],{(1,2,3),(3,4,5)})
print('R')
print(R)

R
 A| B| C|
--|--|--|
 3| 4| 5|
 1| 2| 3|


Raise <b>Column counts not match</b> error when the column counts of two tables do not match.

In [29]:
R | S

OperationNotSupported: Column counts not match

## Diffference

In [30]:
R = Relation('R',['A','B'],{(1,2),(3,4)})
S = Relation('S',['C','D'],{(5,6),(3,4)})
print('R')
print(R)
print('\nS')
print(S)

R
 A| B|
--|--|
 1| 2|
 3| 4|

S
 C| D|
--|--|
 5| 6|
 3| 4|


Raise <b>Cannot subtract different schema</b> error when the two tables have different column names.

In [31]:
R - S

OperationNotSupported: Cannot substract different schema

Change the column name of relation <b>S</b> into <b>A</b> and <b>B</b> to be matched with relation <b>R</b>.

In [32]:
S.rename({'C':'A','D':'B'})
print('S')
print(S)

S
 A| B|
--|--|
 5| 6|
 3| 4|


In [33]:
print(R - S)
(R - S).datalist

 A| B|
--|--|
 1| 2|


[{'A': 1, 'B': 2}]

Add new column to relation <b>R</b>. Now relation <b>R</b> have <b>3 columns</b> and relation <b>S</b> have only <b>2 columns</b>.

In [34]:
R = Relation('R',['A','B','C'],{(1,2,3),(3,4,5)})
print('R')
print(R)

R
 A| B| C|
--|--|--|
 3| 4| 5|
 1| 2| 3|


Raise <b>Column counts not match</b> error when the column counts of two tables do not match.

In [35]:
R - S

OperationNotSupported: Column counts not match

## Cartesian product

In [36]:
R = Relation('R',['A','B'],{(1,2),(3,4)})
S = Relation('S',['C','D'],{(5,6),(7,8)})
print('R')
print(R)
print('\nS')
print(S)

R
 A| B|
--|--|
 1| 2|
 3| 4|

S
 C| D|
--|--|
 5| 6|
 7| 8|


In [37]:
print(R * S)
(R * S).datalist

 A| B| C| D|
--|--|--|--|
 1| 2| 5| 6|
 1| 2| 7| 8|
 3| 4| 5| 6|
 3| 4| 7| 8|


[{'A': 1, 'B': 2, 'C': 5, 'D': 6},
 {'A': 1, 'B': 2, 'C': 7, 'D': 8},
 {'A': 3, 'B': 4, 'C': 5, 'D': 6},
 {'A': 3, 'B': 4, 'C': 7, 'D': 8}]

Create new relation <b>S</b> with one same column name of relation <b>R</b>. That will prefixed the same column name with the name of the relation it comes from followed by a dot.

In [38]:
S = Relation('S',['B','C'],{(5,6),(7,8)})
print('S')
print(S)

S
 B| C|
--|--|
 5| 6|
 7| 8|


In [39]:
print(R * S)
(R * S).datalist

   A| R.B|   C| S.B|
----|----|----|----|
   1|   2|   6|   5|
   1|   2|   8|   7|
   3|   4|   6|   5|
   3|   4|   8|   7|


[{'A': 1, 'R.B': 2, 'C': 6, 'S.B': 5},
 {'A': 1, 'R.B': 2, 'C': 8, 'S.B': 7},
 {'A': 3, 'R.B': 4, 'C': 6, 'S.B': 5},
 {'A': 3, 'R.B': 4, 'C': 8, 'S.B': 7}]

In [40]:
S = Relation('S',['A','B'],{(5,6),(7,8)})
print('S')
print(S)

S
 A| B|
--|--|
 5| 6|
 7| 8|


In [41]:
print(R * S)
(R * S).datalist

 R.A| R.B| S.A| S.B|
----|----|----|----|
   1|   2|   5|   6|
   1|   2|   7|   8|
   3|   4|   5|   6|
   3|   4|   7|   8|


[{'R.A': 1, 'R.B': 2, 'S.A': 5, 'S.B': 6},
 {'R.A': 1, 'R.B': 2, 'S.A': 7, 'S.B': 8},
 {'R.A': 3, 'R.B': 4, 'S.A': 5, 'S.B': 6},
 {'R.A': 3, 'R.B': 4, 'S.A': 7, 'S.B': 8}]